#  sklearn


## Ejemplo de pipeline con ramdom forest

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Supongamos que tienes un DataFrame llamado 'df_1' con las características en X y la variable objetivo en y

# Paso 1: Definir el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Paso 1: Escalado de características
    ('regressor', RandomForestRegressor())  # Paso 2: Modelo de Random Forest
])

# Paso 2: Dividir el DataFrame en características (X) y variable objetivo (y)
X = df_2.drop(['Life expectancy'], axis=1)  # Características
y = df_2['Life expectancy']  # Variable objetivo


# Paso 3: Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Paso 5: Evaluar el rendimiento del modelo
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
